##Intro to Programming Project: Protein TAG Variant Screen

In [6]:
#@title Install dependencies
%%bash -s $use_amber $use_templates

set -e

USE_AMBER=$1
USE_TEMPLATES=$2

if [ ! -f COLABFOLD_READY ]; then
  # install dependencies
  # We have to use "--no-warn-conflicts" because colab already has a lot preinstalled with requirements different to ours
  pip install -q --no-warn-conflicts "colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold"
  # high risk high gain
  pip install -q "jax[cuda11_cudnn805]>=0.3.8,<0.4" -f https://storage.googleapis.com/jax-releases/jax_releases.html
  touch COLABFOLD_READY
fi

# setup conda
if [ ${USE_AMBER} == "True" ] || [ ${USE_TEMPLATES} == "True" ]; then
  if [ ! -f CONDA_READY ]; then
    wget -qnc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local 2>&1 1>/dev/null
    rm Miniconda3-latest-Linux-x86_64.sh
    touch CONDA_READY
  fi
fi
# setup template search
if [ ${USE_TEMPLATES} == "True" ] && [ ! -f HH_READY ]; then
  conda install -y -q -c conda-forge -c bioconda kalign2=2.04 hhsuite=3.3.0 python=3.7 2>&1 1>/dev/null
  touch HH_READY
fi
# setup openmm for amber refinement
if [ ${USE_AMBER} == "True" ] && [ ! -f AMBER_READY ]; then
  conda install -y -q -c conda-forge openmm=7.5.1 python=3.7 pdbfixer 2>&1 1>/dev/null
  touch AMBER_READY
fi

In [7]:
#Loading all necessary modules/packages
#Input protein sequence(s)
from google.colab import files
import os.path
import re
import hashlib
import random

#Run Prediction
import sys
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from colabfold.download import download_alphafold_params, default_data_dir
from colabfold.utils import setup_logging
from colabfold.batch import get_queries, run, set_model_type
K80_chk = !nvidia-smi | grep "Tesla K80" | wc -l
if "1" in K80_chk:
  print("WARNING: found GPU Tesla K80: limited to total length < 1000")
  if "TF_FORCE_UNIFIED_MEMORY" in os.environ:
    del os.environ["TF_FORCE_UNIFIED_MEMORY"]
  if "XLA_PYTHON_CLIENT_MEM_FRACTION" in os.environ:
    del os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]

from colabfold.colabfold import plot_protein
from pathlib import Path
import matplotlib.pyplot as plt

os.mkdir("PDBs")

FileExistsError: ignored

In [8]:
#Functional approach to iterating over given dictionary of values

#Hash function for producing keys for appending to jobname
def add_hash(x,y):
  return x+"_"+hashlib.sha1(y.encode()).hexdigest()[:5]

#Takes a dictionary of values and builds an array of all the filepaths
def input_dict(variant_dict):
  paths_dict = {}
  for name in variant_dict:
    # Initialization of the query_sequence
    # Removes whitespaces
    query_sequence = variant_dict[name]
    query_sequence = "".join(query_sequence.split())
    # Initialization of the jobname
    # Removes whitespaces and append hash key
    jobname = name
    basejobname = "".join(jobname.split())
    basejobname = re.sub(r'\W+', '', basejobname)
    jobname = add_hash(basejobname, query_sequence)
    while os.path.isfile(f"{jobname}.csv"):
      jobname = add_hash(basejobname, ''.join(random.sample(query_sequence,len(query_sequence))))

    with open(f"{jobname}.csv", "w") as text_file:
      text_file.write(f"id,sequence\n{jobname},{query_sequence}")

    #local variables needed to run prediction (most of these are adjustable, but for our purposes we are running them as default)
    #MS
    queries_path=f"{jobname}.csv"
    use_amber = False
    template_mode = "none"
    custom_template_path = None
    use_templates = False
    msa_mode = "MMseqs2 (UniRef+Environmental)"
    pair_mode = "unpaired+paired"
    a3m_file = f"{jobname}.a3m"
    model_type = "auto"
    num_recycles = 3
    save_to_google_drive = False
    dpi = 200

    input_array = [queries_path,use_amber,template_mode,custom_template_path,use_templates,msa_mode,pair_mode,a3m_file,model_type,num_recycles,save_to_google_drive,dpi]

    if use_amber and '/usr/local/lib/python3.7/site-packages/' not in sys.path:
      sys.path.insert(0, '/usr/local/lib/python3.7/site-packages/')

    jobname_path = run_prediction(jobname,input_array,"PDBs")
    
    paths_dict[name] = jobname_path
  return paths_dict

def prediction_callback(unrelaxed_protein, length, prediction_result, input_features, type):
  fig = plot_protein(unrelaxed_protein, Ls=length, dpi=150)
  plt.show()
  plt.close()

#Runs a prediction and moves the resulting rank 1 pdb structure (model does not matter) into a desired directory
#Must provide the data array (contains all the parameters to run prediction) and directory path. 
#Returns the filepath of the PDB structure ofr the variant
def run_prediction(jobname,data_array,directory_name):
  #
  result_dir="."
  if 'logging_setup' not in globals():
      setup_logging(Path(".").joinpath("log.txt"))
      logging_setup = True
  
  queries_path=data_array[0]
  use_amber = data_array[1]
  template_mode = data_array[2]
  custom_template_path = data_array[3]
  use_templates = data_array[4]
  msa_mode = data_array[5]
  pair_mode = data_array[6]
  a3m_file = data_array[7]
  model_type = data_array[8]
  num_recycles = data_array[9]
  save_to_google_drive = data_array[10]
  dpi = data_array[11]
  
  queries, is_complex = get_queries(queries_path)
  model_type = set_model_type(is_complex, model_type)
  download_alphafold_params(model_type, Path("."))
  run(
      queries=queries,
      result_dir=result_dir,
      use_templates=use_templates,
      custom_template_path=custom_template_path,
      use_amber=use_amber,
      msa_mode=msa_mode,    
      model_type=model_type,
      num_models=5,
      num_recycles=num_recycles,
      model_order=[1, 2, 3, 4, 5],
      is_complex=is_complex,
      data_dir=Path("."),
      keep_existing_results=False,
      recompile_padding=1.0,
      rank_by="auto",
      pair_mode=pair_mode,
      stop_at_score=float(100),
      prediction_callback=prediction_callback,
      dpi=dpi
  )

  ex_end_path = ""

  for index in range(1,6):
    end_file = "_unrelaxed_rank_1_model_"+str(index)+".pdb"
    test_path = "/content/" + jobname + end_file
    if (os.path.exists(test_path)):
      ex_end_path = end_file

  old_path = "/content/" + jobname + ex_end_path
  new_path = "/content/"+ directory_name +"/" + jobname + ex_end_path
  os.replace(old_path,new_path)

  return new_path

In [11]:
test_input = {'Protein0': 'EEMQRR', 'Protein0_6X His': 'EEMQRRHHHHHH'}

files = input_dict(test_input)

2022-09-21 07:29:17,377 Query 1/1: Protein0_e969a (length 6)


COMPLETE: 100%|██████████| 150/150 [elapsed: 00:00 remaining: 00:00]


2022-09-21 07:29:17,946 Running model_1


KeyboardInterrupt: ignored

In [10]:
print(files)

{'Protein0': '/content/PDBs/Protein0_6e630_unrelaxed_rank_1_model_4.pdb', 'Protein0_6X His': '/content/PDBs/Protein0_6XHis_02521_unrelaxed_rank_1_model_1.pdb'}
